In [1]:
import numpy as np
import random
from wordPreprocess import WordPreprocess
from gensim.models import word2vec

In [2]:
programs_questions = []
num_lines = 0
with open('program_quesiton2.txt', 'r') as inFile:
    for line in inFile:
        programs_questions.append(line.strip('\n'))
        num_lines += 1
        
        if num_lines % 100000 == 0:
            print ("Finished %d of lines" % num_lines)

word2vec_model = word2vec.Word2Vec.load('word2vec_model2.bin')

Finished 100000 of lines
Finished 200000 of lines
Finished 300000 of lines
Finished 400000 of lines
Finished 500000 of lines
Finished 600000 of lines
Finished 700000 of lines
Finished 800000 of lines
Finished 900000 of lines
Finished 1000000 of lines
Finished 1100000 of lines
Finished 1200000 of lines
Finished 1300000 of lines
Finished 1400000 of lines
Finished 1500000 of lines
Finished 1600000 of lines
Finished 1700000 of lines
Finished 1800000 of lines
Finished 1900000 of lines
Finished 2000000 of lines
Finished 2100000 of lines
Finished 2200000 of lines
Finished 2300000 of lines
Finished 2400000 of lines


word_preprocess = WordPreprocess()
doc_hash = word_preprocess.hash_transform(programs_questions, num_feature=2**7)
print (doc_hash.shape)

In [3]:
doc_vec = []

for sentence in programs_questions:
    words = sentence.split(' ')[:-1]
    vec = np.zeros(shape=(250,), dtype=float)
    count = 0
    for w in words:
        try:
            vec += word2vec_model.wv[w]
            count += 1
        except KeyError:
            pass
        
    #if (count > 0):
    #    vec /= count
    doc_vec.append(vec)

In [4]:
programs = programs_questions[:2396906]
questions = programs_questions[2396906:]

programs_vec = doc_vec[:2396906]
questions_vec = doc_vec[2396906:]

In [11]:
def generate_training_data():
    Xs, Ys = [], []
    NUM_TRAIN = 500000
    itr = 0
    
    while (itr < NUM_TRAIN):
        pos_or_neg = random.randint(0,1)
        
        if pos_or_neg is 1:
            line_id = random.randint(0, len(programs)-1)
            
            L1 = programs[line_id].split(' ')[:-1]
            L2 = programs[line_id+1].split(' ')[:-1]
            
            if (not not L1 and not not L2):
                try:
                    Ys.append(word2vec_model.n_similarity(L1, L2))
                    Xs.append([programs_vec[line_id], programs_vec[line_id+1]])
                    itr += 1
                except KeyError:
                    pass
            
        else:
            line1_id = random.randint(0, len(programs)-1)
            line2_id = random.randint(0, len(programs)-1)
            
            L1 = programs[line1_id].split(' ')[:-1]
            L2 = programs[line2_id].split(' ')[:-1]
            
            if (not not L1 and not not L2):
                try:
                    Ys.append(word2vec_model.n_similarity(L1, L2))
                    Xs.append([programs_vec[line1_id], programs_vec[line2_id]])
                    itr += 1
                except KeyError:
                    pass
            
    return Xs, Ys

In [12]:
from sklearn.model_selection import train_test_split

X, y = generate_training_data()

X_ = []

for t in X:
    X_.append(t[0] + t[1])
    

X_ = np.stack(X_, axis=0)
print (X_)

X_train, X_valid, y_train, y_valid = train_test_split(X_, y, test_size=0.3, random_state=1)

[[ -2.31382322  -0.44065855   0.21759067 ...,  -0.43714296   0.31325008
    3.50430579]
 [ -7.46128528  -2.08195015  -0.69110731 ...,  -4.67145798  -2.13019301
   -1.65524331]
 [-13.63056149  -3.83302283  -3.01976722 ...,  -8.44428488  -9.33636402
    2.20747711]
 ..., 
 [ -1.12300221   1.06858695  -1.61439244 ...,  -2.55131452   2.0256753
   -2.1459115 ]
 [ -4.3073566   -0.13796829   2.30420542 ...,  -0.87576938  -5.03541787
    3.69778971]
 [ -0.52798342  -0.9905298    0.09281074 ...,  -0.18663038   0.50097048
    0.29843731]]


In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_valid_std = sc.transform(X_valid)

In [14]:
from sklearn.linear_model import LinearRegression

regr = LinearRegression()
regr.fit(X_train_std, y_train)

y_pred = regr.predict(X_valid_std)
print("MSE: %f" % mean_squared_error(y_valid, y_pred))

MSE: 0.022058


In [15]:
from sklearn.linear_model import Ridge
lr_rg = Ridge(alpha=1000)
lr_rg.fit(X_train_std, y_train)
    
y_pred_rg = lr_rg.predict(X_valid_std)
    
print ("MSE: %f" % mean_squared_error(y_valid, y_pred_rg))

MSE: 0.022085


from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svm_rbf = SVC(kernel='rbf', random_state=0, C=10.0, gamma=0.2)
svm_rbf.fit(X_train_std, y_train)

y_pred = svm_rbf.predict(X_valid_std)

print ("Accuracy: %.3f" % (accuracy_score(y_valid, y_pred)))

In [16]:
import sys

q_test = []
for q_id in range(500):
    question = questions_vec[q_id*7]
    answer = 0
    sim_max = sys.float_info.min
    for i in range(1,7):
        ans = questions_vec[q_id*7 + i]
        #set_ = [x+y for x, y in zip(question, ans)]
        set_ = question + ans
        #set_ = np.array(set_)
        result = regr.predict(set_.reshape(1,-1))
        if result > sim_max:
            answer = i-1
            sim_max = result
        
    q_test.append((q_id, answer))

In [17]:
print (q_test)

[(0, 4), (1, 0), (2, 3), (3, 0), (4, 0), (5, 3), (6, 2), (7, 1), (8, 0), (9, 4), (10, 0), (11, 2), (12, 3), (13, 0), (14, 2), (15, 0), (16, 4), (17, 4), (18, 0), (19, 3), (20, 0), (21, 1), (22, 4), (23, 0), (24, 3), (25, 1), (26, 2), (27, 2), (28, 3), (29, 2), (30, 5), (31, 2), (32, 3), (33, 2), (34, 4), (35, 3), (36, 4), (37, 4), (38, 1), (39, 3), (40, 3), (41, 4), (42, 4), (43, 3), (44, 3), (45, 5), (46, 4), (47, 1), (48, 3), (49, 3), (50, 5), (51, 2), (52, 5), (53, 4), (54, 2), (55, 1), (56, 3), (57, 1), (58, 0), (59, 3), (60, 0), (61, 0), (62, 3), (63, 0), (64, 4), (65, 4), (66, 4), (67, 4), (68, 3), (69, 1), (70, 1), (71, 4), (72, 0), (73, 3), (74, 4), (75, 4), (76, 1), (77, 5), (78, 1), (79, 3), (80, 5), (81, 3), (82, 2), (83, 5), (84, 3), (85, 1), (86, 2), (87, 2), (88, 2), (89, 1), (90, 0), (91, 4), (92, 2), (93, 5), (94, 5), (95, 1), (96, 1), (97, 5), (98, 3), (99, 5), (100, 2), (101, 0), (102, 0), (103, 1), (104, 0), (105, 5), (106, 5), (107, 2), (108, 3), (109, 1), (110, 4),

In [18]:
import pandas as pd

df = pd.DataFrame(data=q_test, columns=['Id', 'Answer']).to_csv('Answer.csv', index=False)